In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16


In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_epochs = 10
batch_size = 40
learning_rate = 0.0001



In [3]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for VGG input
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# Data Loaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 170M/170M [00:04<00:00, 42.0MB/s]


In [4]:
# Load pre-trained VGG16 and modify classifier
model = vgg16(pretrained=True)

# Freeze the first few layers
for param in model.features[:10].parameters():
    param.requires_grad = False

# Modify the classifier for 10 classes
model.classifier[6] = nn.Linear(4096, 10)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 79.2MB/s]


In [5]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")



Epoch 1/10
Epoch [1/10], Loss: 0.4661
Epoch 2/10
Epoch [2/10], Loss: 0.2180
Epoch 3/10
Epoch [3/10], Loss: 0.1373
Epoch 4/10
Epoch [4/10], Loss: 0.1027
Epoch 5/10
Epoch [5/10], Loss: 0.0763
Epoch 6/10
Epoch [6/10], Loss: 0.0718
Epoch 7/10
Epoch [7/10], Loss: 0.0611
Epoch 8/10
Epoch [8/10], Loss: 0.0549
Epoch 9/10
Epoch [9/10], Loss: 0.0472
Epoch 10/10
Epoch [10/10], Loss: 0.0441


In [6]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Final Test Accuracy: {100 * correct / total:.2f}%')


Final Test Accuracy: 90.62%
